# Ejemplo K-Means

Análisis de datos sobre usuarios de twitter y su personalidad

## 1. Carga de librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.model_selection import cross_validate
from sklearn.metrics import pairwise_distances_argmin_min
import seaborn as sns

%matplotlib inline

## 2. Carga de datos

In [ ]:
df = pd.read_csv("social_media.csv")
df.head()

In [ ]:
df.categoria.value_counts()

## 3. EDA

In [ ]:
df.describe()

In [ ]:
df.describe(include="object")

In [ ]:
df.info()

Sobre las categorías disponibles nos encontramos los siguiente:
* 1. Actor
* 2. Cantante
* 3. Modelo
* 4. TV
* 5. Radio
* 6. Tecnología
* 7. Deportes
* 8. Política
* 9. Escritor

In [ ]:
# Vemos el número de categorías (usuarios)

df.categoria.value_counts()

In [ ]:
print("*CANTIDAD de datos nulos por columna en el dataframe")
print(df.isnull().sum())
print("----------------------------------")
print("*PORCENTAJE de datos nulos por columna en el dataframe")
print(df.isnull().sum()/len(df)*100)

In [ ]:
df.drop(['categoria'],1).hist()
plt.show()

In [ ]:
sns.pairplot(data=df, 
             hue="categoria", 
             vars=df.columns[df.columns != "categoria"], 
             diag_kind="hist")
plt.show()

In [ ]:
sns.pairplot(df, 
            hue='categoria',
            size=4,
            vars=["op","ex","ag"],
            kind='scatter', 
            diag_kind  = "hist")

## Variables de entrada a K-Means

In [ ]:
X = np.array(df[["op","ex","ag",'ne', 'wordcount']])
y = np.array(df['categoria'])
X.shape

Al clusterizar, trataremos de identificar si existen grandes diferencias entre las categorías de usuarios y, el número de clusters.

## Selección del parámetro K, método Elbow

In [ ]:
# Calculo WSS (Within-Cluster-Sum of Squared Errors) en función del 
# número de clústers k
wss = []
for K in range(2, 15):
    clf = KMeans(init="random", random_state=0, n_clusters=K)
    resultados = cross_validate(clf, X, cv=4)
    wss.append(np.mean(resultados["test_score"]) * -1)

In [ ]:
# Represento el método del codo
plt.plot(np.arange(2, 15), wss)
plt.xlabel("K")
plt.ylabel("WSS")
plt.title("Método del codo para obtener el K óptimo")
plt.show()

Parece que 6 clusters es un buen número

## Realización de K-Means con K-óptimo

In [ ]:
# Ajusto el modelo con el mejor parámetro k=5
clf = KMeans(init="random", random_state=0, n_clusters=6)
clf.fit(X)
clusters = clf.predict(X)

In [ ]:
clusters

Mostramos los centroides

In [ ]:
centroids = clf.cluster_centers_
print(centroids)

In [ ]:
cluster, conteo = np.unique(clusters, return_counts=True)

print(np.asarray((cluster, conteo)).T)

## Obtención de métricas

In [ ]:
df["asignacion_cluster"] = clusters

In [ ]:
df.head()

In [ ]:
df[(df["asignacion_cluster"]) == 1].describe()

In [ ]:
df[(df["asignacion_cluster"]) == 5].describe()

In [ ]:
colores=['red','green','blue','cyan','yellow', "pink"]
asignar=[]
for row in clusters:
    asignar.append(colores[row])

In [ ]:
# Mostramos las dos primeras variables y sus asignaciones
f1 = df['op']
f2 = df['ex']

plt.scatter(f1, f2, c=asignar, s=70)
plt.scatter(clf.cluster_centers_[:, 0], 
            clf.cluster_centers_[:, 1], 
            marker='*', c=colores, 
            s=1000)
plt.show()